In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from small_script.myFunctions import *
import feather
import Bio.PDB as bio

from sklearn.metrics import confusion_matrix


d3_to_index = bio.Polypeptide.d3_to_index  # we may want to adjust this in the future.
three_to_one = bio.Polypeptide.three_to_one
one_to_index = bio.Polypeptide.one_to_index

%matplotlib inline
%load_ext autoreload
%autoreload 2
plt.rcParams['figure.figsize'] = [16.18033, 10]

In [2]:

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.datasets import imdb
from keras.utils import to_categorical

from keras.models import model_from_json

from keras.metrics import top_k_categorical_accuracy

max_features = 20
batch_size = 1024*2
maxlen = 9
n = int(1e4)



Using TensorFlow backend.


# get test data into csv format

In [3]:
# get the real frag info
min_seq_sep=3
max_seq_sep=9
parser = bio.PDBParser(QUIET=True)
three_to_one = bio.Polypeptide.three_to_one

#         out.write("pdb,i,j,res1,res2,dis_ca_ca,dis_ca_cb,dis_cb_ca,dis_cb_cb\n")


def getFragInfoFromPdb(pdb_list, pre, testpre):
    for cc, p in enumerate(pdb_list):
        has_something = False
        name = p.lower()[:4]
        pdbId = pdb = name
        location = pre+f"{pdbId}/{pdbId}/crystal_structure.pdb"
        structure = parser.get_structure("x", location)
        with open(testpre+f"{pdbId}.csv", "w") as out:
            for model in structure:
                for chain in model:
                    all_residues = list(chain)
                    # print(all_residues)
                    for i, residue in enumerate(all_residues):
                        outLine = ""
                        need = True
                        dis_ca_ca = []
                        dis_ca_cb = []
                        dis_cb_ca = []
                        dis_cb_cb = []
                        resId = residue.get_id()[1]
                        frag = all_residues[i:i+max_seq_sep]
                        resseq_list = [x.get_id()[1] for x in frag]
                        fragSeq = "".join([three_to_one(x.get_resname()) for x in frag])
                        # print(i, fragSeq)
                        if len(frag) != 9:
                            continue
                        if not np.all(np.ediff1d(resseq_list)==1):
                        # print(f"mismatch, {resId}, {resseq_list}")
                            continue
                        for ii in range(7):
                            if not need:
                                break
                            try:
                                r1 = frag[ii]
                            except Exception as ex:
                                need = False
                                break
                            # print(i, residue.get_resname())
                            for j, r2 in enumerate(frag[(min_seq_sep+ii):]):
                                # The H of GLY is replaced with CB in this dataset
                                try:
                                    r2_cb = r2["CB"]
                                except Exception as ex:
                                    try:
                                        r2_cb = r2["CA"]
                                    except Exception as ex:
                                        # print(pdbId, resId)
                                        need = False
                                        break
                                try:
                                    r1_cb = r1["CB"]
                                except Exception as ex:
                                    try:
                                        r1_cb = r1["CA"]
                                    except Exception as ex:
                                        # print(pdbId, resId)
                                        need = False
                                        break
                                try:
                                    r1_ca = r1["CA"]
                                    r2_ca = r2["CA"]
                                except Exception as ex:
                                    print(pdbId, resId)
                                    # os.system(f"echo '{pdbId}' >> {pre}/without_discontinues_and_gly_exception_2")
                                    need = False
                                    break
                                dis_ca_ca.append(str(r1_ca-r2_ca))
                                dis_ca_cb.append(str(r1_ca-r2_cb))
                                dis_cb_ca.append(str(r1_cb-r2_ca))
                                dis_cb_cb.append(str(r1_cb-r2_cb))
                        if need:
                            outLine = f"{pdbId},{i},{fragSeq},"+",".join(dis_ca_ca)+\
                            ","+",".join(dis_ca_cb)+\
                            ","+",".join(dis_cb_ca)+\
                            ","+",".join(dis_cb_cb)+"\n"
                            out.write(outLine)



In [35]:
pre = "/Users/weilu/Research/server/jan_2019/iterative_optimization_another_set/all_simulations/"
testpre = "/Users/weilu/Research/optimization/fragment/testSet/"
pdb_list = "1FC2C, 1ENH, 2GB1, 2CRO, 1CTF, 4ICB".split(", ")
getFragInfoFromPdb(pdb_list, pre, testpre)


headline = "pdb,i,seq," + ",".join([f"caca_{i}" for i in range(1,22)]) + \
        "," + ",".join([f"cacb_{i}" for i in range(1,22)]) + \
        "," + ",".join([f"cbca_{i}" for i in range(1,22)]) + \
        "," + ",".join([f"cbcb_{i}" for i in range(1,22)]) 

# combine all to one data.
folder = "testSet"
combined_csv = "test_data"
os.system(f"echo '{headline}' > /Users/weilu/Research/optimization/fragment/{combined_csv}.csv")
pdb_list = os.listdir(f"/Users/weilu/Research/optimization/fragment/{folder}/")
for cc, pdb in enumerate(pdb_list):
    os.system(f"cat /Users/weilu/Research/optimization/fragment/{folder}/{pdb} >> /Users/weilu/Research/optimization/fragment/{combined_csv}.csv")



# pipline from test data to accuracy

In [4]:
data_original = pd.read_csv("/Users/weilu/Research/optimization/fragment/test_data.csv")

In [19]:
import pickle
# pickle.dump(kmeans, open("kmeans_cluster100_v2", "wb"))
kmeans = pickle.load(open("/Users/weilu/Research/optimization/fragment/kmeans_cluster100_v2_2", "rb"))

In [20]:
data_original["cluster"] = kmeans.predict(data_original.iloc[:, 3:87].values)
def getScore(data, km):
    return np.sqrt(((km.cluster_centers_[int(data.values[-1])] - data.values[:-1])**2).sum())
data_original["rmsd"] = data_original.iloc[:,3:88].apply(lambda x: getScore(x, kmeans), axis=1)

In [21]:
data = data_original[["pdb", "i", "seq","cluster", "rmsd"]].reset_index(drop=True)
data["cluster"] = data["cluster"].astype(int)
for i in range(1,10):
    data[f"s{i}"] = data["seq"].apply(lambda x: one_to_index(x[i-1]))

In [28]:
# pre = "/Users/weilu/Research/server/jan_2019/lstm100_v3/"
pre = "/Users/weilu/Research/server/feb_2019/lstm100/"
json_file = open(f"{pre}/model.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(f"{pre}/model.h5")

In [29]:
x_test = data.iloc[:, 5:14].values
y_test_value = data["cluster"].values
y_test = to_categorical(np.array(y_test_value), num_classes=100)

In [30]:
def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)
def top_5_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=5)
def top_10_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=10)
def top_20_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=20)
loaded_model.compile('adam', 'categorical_crossentropy', metrics=['accuracy', top_3_accuracy, 
                                                                  top_5_accuracy, 
                                                                  top_10_accuracy, top_20_accuracy])

In [31]:
top_n = 2
all_frag = []
for fragSeq in data_original["seq"]:
    fragIndex = [one_to_index(x) for x in fragSeq]
    all_frag.append(fragIndex)
all_frag = np.array(all_frag)
predict_prob = loaded_model.predict(all_frag)
clusters = np.argsort(-predict_prob)[:, :top_n]

In [32]:
data_original = data_original.assign(pred_top1=clusters[:,0], pred_top2=clusters[:,1])

In [33]:
loaded_model.evaluate(x_test, y_test)

315/315 [==============================] - 0s 1ms/step


[2.9926906101287356,
 0.3238095232891658,
 0.5238095233364711,
 0.6285714295175341,
 0.7904761910438538,
 0.8952380939135476]

In [15]:
loaded_model.evaluate(x_test, y_test)

315/315 [==============================] - 0s 162us/step


[2.9494901664673336,
 0.24761904769000553,
 0.4571428561021411,
 0.5904761914222959,
 0.7333333344686599,
 0.8666666691265409]

In [119]:
loaded_model.evaluate(x_test, y_test)

315/315 [==============================] - 0s 928us/step


[3.0338871017334954,
 0.2730158725428203,
 0.46984127012510146,
 0.561904764175415,
 0.6793650791758583,
 0.8253968278566997]

In [35]:
data_original

,pdb,i,seq,caca_1,caca_2,caca_3,caca_4,caca_5,caca_6,caca_7,...,cbcb_16,cbcb_17,cbcb_18,cbcb_19,cbcb_20,cbcb_21,cluster,rmsd,pred_top1,pred_top2
0,1fc2,0,FNKEQQNAF,8.888883,8.460715,7.119914,9.647429,12.018316,11.133865,7.348096,...,4.078050,5.380944,8.754528,5.897926,6.094513,4.603702,19,11.223527,19,70
1,1fc2,1,NKEQQNAFY,7.348096,7.473793,9.262225,11.476255,11.715269,12.939787,6.442751,...,5.897926,6.094513,9.761956,4.603702,6.167987,5.783719,70,8.539869,23,67
2,1fc2,2,KEQQNAFYE,6.442751,7.414185,8.963633,10.192899,11.834385,13.640175,4.401744,...,4.603702,6.167987,9.624481,5.783719,6.944193,5.960988,55,7.491690,94,60
3,1fc2,3,EQQNAFYEI,4.401744,5.670711,7.847085,9.221101,10.337097,11.848258,5.228988,...,5.783719,6.944193,9.913838,5.960988,6.294312,6.283356,55,4.934995,81,0
4,1fc2,4,QQNAFYEIL,5.228988,5.918028,8.641494,10.228614,10.586554,11.926364,4.353926,...,5.960988,6.294312,9.291823,6.283356,5.991059,5.657963,55,3.384191,66,17
5,1fc2,5,QNAFYEILH,4.353926,5.774894,8.417571,9.503526,9.825358,11.926640,5.178828,...,6.283356,5.991059,9.782140,5.657963,6.319229,6.061292,55,3.406676,70,19
6,1fc2,6,NAFYEILHL,5.178828,6.544211,8.836107,9.849736,10.846585,12.570953,5.361785,...,5.657963,6.319229,9.861423,6.061292,6.468529,5.591717,55,3.127405,38,89
7,1fc2,7,AFYEILHLP,5.361785,6.228668,8.385612,9.943428,10.675358,14.441677,5.407273,...,6.061292,6.468529,10.820477,5.591717,11.155780,10.948488,48,8.780100,8,71
8,1fc2,8,FYEILHLPN,5.407273,6.014185,8.673256,10.108251,13.722568,15.214639,5.043434,...,5.591717,11.155780,11.494132,10.948488,13.046161,10.816037,23,6.889736,23,72
9,1fc2,9,YEILHLPNL,5.043434,6.241478,8.726650,11.999007,14.309925,12.614616,4.949817,...,10.948488,13.046161,8.840693,10.816037,8.836898,3.394289,56,8.923290,56,99


In [ ]:
pdb = "1ctf"
top_n = 2
seqFile = f"/Users/weilu/Research/server/jan_2019/iterative_optimization_another_set/all_simulations/{pdb}/{pdb}/{pdb}.seq"
with open(seqFile) as f:
    lines = f.readlines()
a = lines[0].strip()
all_frag = []
for i in range(0, len(a)-8):
    frag = a[i:i+9]
    # print(frag)
    fragIndex = [one_to_index(x) for x in frag]
    # print(fragIndex)
    all_frag.append(fragIndex)
all_frag = np.array(all_frag)
predict_prob = loaded_model.predict(all_frag)
clusters = np.argsort(-predict_prob)[:, :top_n]
# cluster100_centers.to_feather("/Users/weilu/Research/optimization/fragment/cluster_centers.feather")
cluster100_centers = feather.read_dataframe("/Users/weilu/Research/optimization/fragment/cluster_centers.feather")

In [77]:
fragSeq = "NEEQRNGFI"
predict_prob = loaded_model.predict(np.array([one_to_index(x) for x in fragSeq]).reshape(1,9))
np.argsort(-predict_prob)[:, :10]

array([[16, 86, 48, 23, 55,  1, 62, 58, 12, 99]])

In [67]:
top_n = 2
all_frag = []
for fragSeq in data_original["seq"]:
    fragIndex = [one_to_index(x) for x in fragSeq]
    all_frag.append(fragIndex)
all_frag = np.array(all_frag)
predict_prob = loaded_model.predict(all_frag)
clusters = np.argsort(-predict_prob)[:, :top_n]

In [73]:
data_original = data_original.assign(pred_top1=clusters[:,0], pred_top2=clusters[:,1])

In [78]:
from keras.metrics import top_k_categorical_accuracy

In [85]:
data.head()

,pdb,i,seq,cluster,rmsd,s1,s2,s3,s4,s5,s6,s7,s8,s9
0,1bg6A02,107,RAVNVPTPL,0,5.756177,14,0,17,11,17,12,16,12,9
1,3g85A02,38,HKNGIKISE,0,6.329374,6,8,11,5,7,8,7,15,3
2,4je5C00,166,EAQGVITFP,0,6.365145,3,0,13,5,17,7,16,4,12
3,3q41A01,88,GFYRIPVLG,0,6.389094,5,4,19,14,7,12,17,9,5
4,1auqA00,144,KKKKVIVIP,0,6.450880,8,8,8,8,17,7,17,7,12


In [65]:

np.argsort(-predict_prob)[:, :top_n]

array([[19, 12]])

In [ ]:
pdb = "1ctf"

seqFile = f"/Users/weilu/Research/server/jan_2019/iterative_optimization_another_set/all_simulations/{pdb}/{pdb}/{pdb}.seq"
with open(seqFile) as f:
    lines = f.readlines()
a = lines[0].strip()
all_frag = []
for i in range(0, len(a)-8):
    frag = a[i:i+9]
    # print(frag)
    fragIndex = [one_to_index(x) for x in frag]
    # print(fragIndex)
    all_frag.append(fragIndex)
all_frag = np.array(all_frag)
predict_prob = loaded_model.predict(all_frag)
clusters = np.argsort(-predict_prob)[:, :top_n]